In [ ]:
conda install pyspark


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyspark

In [ ]:
import findspark
findspark.init()


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


In [ ]:
spark

In [ ]:
from pyspark.sql.types import *

In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
carros_temp = spark.read.csv("/home/cliente/download/Carros.csv", inferSchema=True, header=True, sep=";")
carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [ ]:
carros = carros_temp.select("consumo", "cilindros", "cilindradas", "hp")
carros.show(5)

+-------+---------+-----------+---+
|consumo|cilindros|cilindradas| hp|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [ ]:
veccaracteristicas = VectorAssembler(inputCols=[("consumo"), ("cilindros"), ("cilindradas")], outputCol="caracteristicas")

In [ ]:
carros = veccaracteristicas.transform(carros)

In [ ]:
carros.show(5)

+-------+---------+-----------+---+-----------------+
|consumo|cilindros|cilindradas| hp|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [ ]:
carrosTreino, carrosTeste = carros.randomSplit([0.7,0.3])

In [ ]:
print(carrosTreino.count())

21


In [ ]:
print(carrosTeste.count())

11


In [ ]:
reglin = LinearRegression(featuresCol="caracteristicas",labelCol="hp")
modelo = reglin.fit(carrosTreino)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f9140a16e60>
Traceback (most recent call last):
  File "/home/cliente/anaconda3/lib/python3.7/site-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LinearRegression' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f9140a16e60>
Traceback (most recent call last):
  File "/home/cliente/anaconda3/lib/python3.7/site-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LinearRegression' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f9140a16e60>
Traceback (most recent call last):
  File "/home/cliente/anaconda3/lib/python3.7/site-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
Attrib

In [ ]:
previsao = modelo.transform(carrosTeste)

In [ ]:
previsao.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|consumo|cilindros|cilindradas| hp|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|238.94617278700588|
|     21|        6|        160|110| [21.0,6.0,160.0]|238.94617278700588|
|     26|        4|       1203| 91|[26.0,4.0,1203.0]| 185.7765010586709|
|    133|        8|        350|245|[133.0,8.0,350.0]|219.75249140225156|
|    155|        8|        318|150|[155.0,8.0,318.0]|209.10245426804278|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [ ]:
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="hp", metricName="rmse")

Exception ignored in: <function JavaWrapper.__del__ at 0x7f9140a16e60>
Traceback (most recent call last):
  File "/home/cliente/anaconda3/lib/python3.7/site-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LinearRegression' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f9140a16e60>
Traceback (most recent call last):
  File "/home/cliente/anaconda3/lib/python3.7/site-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LinearRegression' object has no attribute '_java_obj'


In [ ]:
rmse = avaliar.evaluate(previsao)

In [ ]:
print(rmse)

68.90558239826922


In [ ]:
rfreg = RandomForestRegressor(featuresCol="caracteristicas", labelCol="hp")


In [ ]:
modelo2 = rfreg.fit(carrosTreino)

In [ ]:
previsao2 = modelo2.transform(carrosTeste)

In [ ]:
previsao2.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|consumo|cilindros|cilindradas| hp|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|149.21855644355645|
|     21|        6|        160|110| [21.0,6.0,160.0]|149.21855644355645|
|     26|        4|       1203| 91|[26.0,4.0,1203.0]|  129.314010989011|
|    133|        8|        350|245|[133.0,8.0,350.0]|218.07815919374738|
|    155|        8|        318|150|[155.0,8.0,318.0]|196.89861373920195|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [ ]:
rmse2 = avaliar.evaluate(previsao2)

In [ ]:
print(rmse2)

35.718824322565204


In [ ]:
print(rmse)

68.90558239826922
